In [9]:
from market_analysis.models import Candle, UserProfile, Symbol
# from market_analysis import views
from heystox_intraday.select_stocks_for_trading import *
import pandas as pd
# from heystox_trade.settings import *
from datetime import datetime, timedelta

In [10]:
symbols = Symbol.objects.all()
for symbol in symbols:
    print(symbol.is_stock_ohl(date="24/09/2019"))

AppRegistryNotReady: Models aren't loaded yet.

In [7]:
# obj = Candle.objects.all().values()
# # new_header = ["Exchange", "Token", "NA", "Symbol", "Full Name", "closed_price", "NA1", "NA2", "Tick Size", "Lot Size", "Instrument Type", "ISIN"]
# df = pd.DataFrame(obj)
# df = df["open_price"]
# # obj.columns = new_header
# # new_obj = obj[(obj.closed_price >= 55) & (obj.closed_price <= 250)]
# # new_obj

207.25